# ChatGLM4聊天机器人

## 环境配置

设置pip源为清华源，或者其他任何一个可用的源

In [1]:
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple

Writing to /home/mindspore/.config/pip/pip.conf


安装mindspore和mindnlp等包

In [2]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.3.0，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
!export MINDSPORE_VERSION=2.3.1
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/aarch64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.mirrors.ustc.edu.cn/simple

In [ ]:
# 查看当前 mindspore 版本
!pip show mindspore

In [2]:
#安装mindnlp的daily包，待正式发布后可改为直接安装mindnlp包
!pip install https://mindspore-courses.obs.cn-north-4.myhuaweicloud.com/mindnlp/mindnlp-0.4.0-py3-none-any.whl -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install mindnlp==0.4.0
!pip install jieba -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tiktoken -i https://pypi.tuna.tsinghua.edu.cn/simple
%env HF_ENDPOINT=https://hf-mirror.com

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 132.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 117.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.1/777.1 kB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 140.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 18.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.6 MB/s eta 0:00:00eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314458 sha256=818667460c1bf04d080dba874c100bb4797f8480c3e9acc0300ce9dfad3b18ed
  Stored in directory: /home/mindspore/.cache/pip/wheels/1a/76/68/b6d79c4db704bb18d54

## 代码开发

In [5]:
import mindspore
from mindnlp.core import no_grad
from mindnlp.transformers import AutoModelForCausalLM, AutoTokenizer
from mindspore._c_expression import _framework_profiler_step_start
from mindspore._c_expression import _framework_profiler_step_end
# 加载tokenizer文件
tokenizer = AutoTokenizer.from_pretrained("ZhipuAI/glm-4-9b-chat", mirror='modelscope')

# 在此输入希望和ChatGLM4对话的内容
query = "你是一个陪伴4到6岁孩子玩耍的智能机器人，现在你正在陪一个小男孩玩讲故事游戏，小朋友向你展示了四只小动物玩具：他们是熊猫，老虎，大象，猫，请你按照固定的出场顺序，以大森林为背景，用这几只小动物为主角给你陪伴的小朋友讲一个引人入胜的童话故事。注意：1. 每个小动物应该有固定的人设，包括性格特征和背景设定，不可更改2. 故事要有波澜，但是整体应该温馨正能量3. 请你注意表达方式，尽可能用小朋友听得懂的话给孩子口述故事"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="ms",
                                       return_dict=True
                                       )
print(inputs)
# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    "ZhipuAI/glm-4-9b-chat",
    mirror='modelscope',
    ms_dtype=mindspore.float16,
).eval()

# 设置采样、最大生成长度等配置，开始推理
# _framework_profiler_step_start()
gen_kwargs = {"max_length": 1000, "do_sample": True, "top_k": 1}
with no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# _framework_profiler_step_end()

{'input_ids': Tensor(shape=[1, 131], dtype=Int64, value=
[[151331, 151333, 151336 ...  99149,  99801, 151337]]), 'attention_mask': Tensor(shape=[1, 131], dtype=Int64, value=
[[1, 1, 1 ... 1, 1, 1]]), 'position_ids': Tensor(shape=[1, 131], dtype=Int64, value=
[[  0,   1,   2 ... 128, 129, 130]])}


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]



在一个遥远的森林里，有一个美丽的大森林，这里住着许多可爱的动物们。今天，我们要讲的故事就是关于大森林里的熊猫、老虎、大象和猫的冒险经历。

故事开始啦！

从前，大森林里有一只憨态可掬的熊猫，它叫小圆。小圆非常喜欢吃竹子，每天都会在竹林里悠闲地吃竹子，它总是笑眯眯的，非常友好。

在熊猫小圆的附近，住着一只威风凛凛的老虎，它叫大虎。大虎是森林里的守护者，它勇敢、聪明，但有时候会因为过于严肃而显得有些凶猛。

大象小象是森林里的大个子，它叫大力。大力非常勤劳，总是帮助其他动物搬运重物，它的皮肤厚厚的，就像一座移动的小山。

还有一只小巧玲珑的猫，它叫小花。小花非常机灵，总是能找到最好的捕猎地点，是森林里的小猎手。

有一天，大森林里来了一只凶猛的野狼，它想要霸占这片土地。小圆、大虎、大力和小花决定联手，保护他们的家园。

首先，勇敢的大虎站了出来，它用它的爪子在地上画了一个大大的圈，告诉野狼：“这是我们的领地，你不能进来！”

野狼瞪大了眼睛，它嘲笑道：“就凭你们这些小家伙，也想阻止我？”

这时，小花跳了出来，它说：“别小看我们，我们可是森林里的精英！”

小花迅速地跳到了树上，找到了一个绝佳的捕猎位置。它瞄准野狼，轻轻地跳下树，用爪子抓住了野狼的尾巴。

大力看到小花行动，立刻用它的长鼻子卷起了一棵大树，向野狼扔去。野狼被大树砸中，摔倒在地。

小圆看到这个机会，立刻跑过去，用它的圆滚滚的身体挡住了野狼的去路。野狼愤怒地咆哮着，但小圆毫不畏惧，它用它的竹子做的竹筒，轻轻地敲打着野狼的头部。

就在这时，大虎趁机冲了过来，用它的爪子猛地抓住了野狼的脖子。小花也跳了下来，用它的爪子帮助大虎制服了野狼。

野狼被他们吓跑了，大森林恢复了往日的宁静。小圆、大虎、大力和小花成为了森林里的英雄，他们一起庆祝胜利。

从此以后，大森林里的动物们过上了幸福快乐的生活，小圆、大虎、大力和小花也成为了最好的朋友。他们一起守护着这片美丽的家园，直到永远。

故事讲完了，小朋友，你觉得这个故事怎么样呢？是不是很温馨又正能量呢？
